In [1]:
data_var = '2023-12-26'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
6952,2023-12-26,China Wcba Feminina,08:30,Xinjiang F,Zhejiang Golden Bulls F,2.65,1.44,152.5,1.85,1.85,3.5,2.08,1.66,r3npcyPa,0.377358,0.694444,0.540541,0.540541,0.071803,130.798,56.400533,0.431203,3.0,0.000000,0.000000,204.61,1.608,0.766955,0.476962,92.0,237.364,203.165906,0.855926,1.8,1.643168,0.912871,89.61,3.040,2.755585,0.906443,5.0,79,87,2.59,1.03,108.162,293.990,0.418386,0.000000,0.158815,1.46,0.292,5.650685,0.000000,0.0,0.000000,-1.13,-0.226,-1.946903,0.000000,0.0,0.000000
6953,2023-12-26,Europa Liga Bnxt,16:30,Mechelen,Limburg,1.72,2.02,146.5,1.87,1.79,-2.5,1.98,1.74,b5FGYyhF,0.581395,0.495050,0.534759,0.558659,0.076445,110.160,42.884039,0.389289,1.2,1.643168,1.369306,107.20,1.494,0.332235,0.222379,-24.0,96.478,9.928183,0.102906,1.8,1.643168,0.912871,92.66,1.280,0.118322,0.092439,37.0,80,82,1.34,1.13,0.000,0.000,0.113440,0.030912,0.091240,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
6954,2023-12-26,Europa Liga Bnxt,16:30,Circus Brussels,Spirou Charleroi,1.58,2.29,141.5,1.80,1.86,-4.5,2.03,1.70,GtQBZHx9,0.632911,0.436681,0.555556,0.537634,0.069593,153.632,26.815028,0.174541,1.2,1.643168,1.369306,174.08,2.222,0.543618,0.244652,-11.0,203.956,113.921499,0.558559,1.2,1.643168,1.369306,125.33,3.052,1.806535,0.591919,-29.0,64,83,2.72,1.51,0.000,0.000,0.259455,0.023184,0.125118,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
6955,2023-12-26,Coreia Do Sul Kbl,07:00,LG Sakers,Anyang JungKwanJang,1.18,4.86,164.5,1.87,1.79,-12.5,2.07,1.67,QRmHsxr6,0.847458,0.205761,0.534759,0.558659,0.053219,110.638,20.857102,0.188517,1.2,1.643168,1.369306,134.68,1.380,0.169558,0.122868,1.0,206.314,95.718046,0.463944,0.0,0.000000,NaN,225.25,2.550,1.102384,0.432307,-88.0,91,85,1.48,2.65,117.094,194.342,0.861640,0.030912,0.151253,-0.90,-0.180,-1.000000,0.714097,0.8,0.085903,-3.58,-0.716,-5.391061,0.497040,0.6,0.102960
6956,2023-12-26,China Wcba Feminina,08:30,Shanghai F,Shanxi Xing Rui F,1.47,2.56,158.5,1.87,1.79,-5.5,2.06,1.68,xQoldev6,0.680272,0.390625,0.534759,0.558659,0.070897,202.816,139.632181,0.688467,1.8,1.643168,0.912871,105.30,2.446,1.742392,0.712343,46.0,114.980,37.511896,0.326247,2.4,1.341641,0.559017,99.84,1.468,0.844020,0.574946,39.0,90,96,1.17,1.04,207.732,156.180,0.382504,0.030912,0.143690,-1.24,-0.248,-1.895161,0.000000,0.0,0.000000,1.59,0.318,4.905660,0.000000,0.0,0.000000
6957,2023-12-26,Cazaquistão Liga Nacional Feminina,04:30,Aktau F,Astana F,7.01,1.08,134.5,1.85,1.85,14.5,2.02,1.70,K4w88kRD,0.142653,0.925926,0.540541,0.540541,0.068579,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,81,0,1.04,0.00,0.000,0.000,1.036624,0.000000,0.121653,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
6958,2023-12-26,China Wcba Feminina,08:30,Beijing F,Inner Mongolia F,5.72,1.12,152.5,1.87,1.79,11.5,2.00,1.72,nygyacfm,0.174825,0.892857,0.534759,0.558659,0.067682,181.418,90.643603,0.499640,1.2,1.643168,1.369306,295.04,2.626,1.562636,0.595063,-29.0,105.500,7.529665,0.071371,3.0,0.000000,0.000000,107.10,1.220,0.149499,0.122540,70.0,64,85,4.61,1.26,92.762,0.000,0.951079,0.030912,0.106446,-2.79,-0.558,-8.458781,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,

## Apostas do dia

### Back Home V1 (QCut)

In [8]:
# Duplicando o dataset
df_bh_v1 = jogos.copy()

df_bh_v1 = df_bh_v1[(df_bh_v1['Odds_H'] >= 1.6) & (df_bh_v1['Odds_H'] <= 2.4)]

if not df_bh_v1.empty:    
    # Filtros pós modelo
    df_bh_v1 = df_bh_v1[((df_bh_v1['CG_A_Last_Game'] < 0.0) | (df_bh_v1['CG_A_Last_Game'] >= 160))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['CV_HA'] < 0.0334) | (df_bh_v1['CV_HA'] >= 1.414))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['P_Diff'] < 0.0464) | (df_bh_v1['P_Diff'] >= 0.065))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['CG_A_Last_Game'] < 348) | (df_bh_v1['CG_A_Last_Game'] >= 920))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['Goals_Last_H'] >= 79) & (df_bh_v1['Goals_Last_H'] <= 110)) | ((df_bh_v1['Goals_Last_H'] > 120.8) & (df_bh_v1['Goals_Last_H'] <= 143))]

    if not df_bh_v1.empty:
        df_bh_v1.to_csv(f'apostas_do_dia/{data_var}/back_home_v1_{data_var}.csv', index=False)

        print('------------ MÉTODO BACK HOME V1 (QCUT) ------------')
        print()

        df_bh_v1['Bet'] = 'Back Home'
        ligs = df_bh_v1['League'].unique().tolist()

        display(df_bh_v1[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO BACK HOME V1 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
6982,00:30,Eua Nba,Phoenix Suns,Dallas Mavericks,1.6,Back Home


### Back Home V2 (LR)

In [9]:
# Duplicando o dataset
df_bh_v2 = jogos.copy()

df_bh_v2 = df_bh_v2[(df_bh_v2['Odds_H'] >= 1.6) & (df_bh_v2['Odds_H'] <= 2.4)]
df_bh_v2.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v2.empty:
    mdl_bh_v2 = load_model('ML/back_home/v2/v2_back_home', verbose=False)
    prev = predict_model(mdl_bh_v2, df_bh_v2)
    prev = prev[prev['prediction_label'] == 1]

    # Filtros pós modelo
    prev = prev[((prev['CV_Odds_Geral_H'] <= 0.224) | (prev['CV_Odds_Geral_H'] > 0.441))]
    prev = prev[((prev['P(U)'] < 0.526) | (prev['P(U)'] > 0.535))]
    prev = prev[((prev['Last_Odd_H'] <= 1.59) | (prev['Last_Odd_H'] > 2.3))]

    if not df_bh_v2.empty:
        df_bh_v2.to_csv(f'apostas_do_dia/{data_var}/back_home_v2_{data_var}.csv', index=False)

        print('------------ MÉTODO BACK HOME V2 (LR) ------------')
        print()

        df_bh_v2['Bet'] = 'Back Home'
        ligs = df_bh_v2['League'].unique().tolist()

        display(df_bh_v2[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO BACK HOME V2 (LR) ------------



,Time,League,Home,Away,Odds_H,Bet
6953,16:30,Europa Liga Bnxt,Mechelen,Limburg,1.72,Back Home
6965,13:40,Israel Liga Leumit,Maccabi Raanana,Elitzur Ashkelon,1.98,Back Home
6979,11:00,Catar Liga De Basquete Do Catar,Al Shamal,Al Sadd,1.61,Back Home
6981,15:15,Áustria Superliga,Wels,Swans Gmunden,1.85,Back Home
6982,00:30,Eua Nba,Phoenix Suns,Dallas Mavericks,1.60,Back Home
6985,22:00,Eua Nba,Chicago Bulls,Atlanta Hawks,1.84,Back Home
6986,22:00,Eua Nba,Houston Rockets,Indiana Pacers,1.72,Back Home
6988,22:00,Eua Nba,Oklahoma City Thunder,Minnesota Timberwolves,1.61,Back Home
6989,22:00,Eua Nba,San Antonio Spurs,Utah Jazz,2.30,Back Home
6990,15:30,Europa Eurocopa,JL Bourg,Gran Canaria,1.80,Back Home


### Under V1 (QCut)

In [10]:
# Duplicando o dataset
df_u_v1 = jogos.copy()

df_u_v1 = df_u_v1[(df_u_v1['Odds_H'] >= 1.6) & (df_u_v1['Odds_H'] <= 2.4)]

if not df_u_v1.empty:    
    # Filtros pós modelo
    df_u_v1 = df_u_v1[((df_u_v1['CG_A_Last_Game'] <= 191) | (df_u_v1['CG_A_Last_Game'] > 280))]
    df_u_v1 = df_u_v1[((df_u_v1['Saldo_Gols_H'] <= 0) | (df_u_v1['Saldo_Gols_H'] > 16))]
    df_u_v1 = df_u_v1[((df_u_v1['CG_H_Last_Game'] <= 166.6) | (df_u_v1['CG_H_Last_Game'] > 260.85))]
    df_u_v1 = df_u_v1[((df_u_v1['Avg_CG_H'] <= 262.034) | (df_u_v1['Avg_CG_H'] > 969.8))]
    df_u_v1 = df_u_v1[((df_u_v1['MediaCustoGolAway'] <= 200.907) | (df_u_v1['MediaCustoGolAway'] > 868.35))]
    df_u_v1 = df_u_v1[((df_u_v1['P_Diff'] <= 0.0747) | (df_u_v1['P_Diff'] > 0.0971))]
    df_u_v1 = df_u_v1[((df_u_v1['HA_Odds_A'] >= 0) & (df_u_v1['HA_Odds_A'] <= 1.68)) | ((df_u_v1['HA_Odds_A'] > 1.8) & (df_u_v1['HA_Odds_A'] <= 1.9))]
    df_u_v1 = df_u_v1[((df_u_v1['CV_HA'] >= 0) & (df_u_v1['CV_HA'] <= 0.162))]
    df_u_v1 = df_u_v1[((df_u_v1['Last_Odd_A'] <= 1.297) | (df_u_v1['Last_Odd_A'] > 1.62))]
    df_u_v1 = df_u_v1[((df_u_v1['P(A)'] <= 0.234) | (df_u_v1['P(A)'] > 0.365))]
    
    if not df_u_v1.empty:
        df_u_v1.to_csv(f'apostas_do_dia/{data_var}/under_v1_{data_var}.csv', index=False)

        print('------------ MÉTODO UNDER V1 (QCUT) ------------')
        print()

        df_u_v1['Bet'] = 'Back Home'
        ligs = df_u_v1['League'].unique().tolist()

        display(df_u_v1[['Time', 'League', 'Home', 'Away', 'Odds_Under', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

Sem jogos


### Back Home V3 (QCut)

In [11]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
6981,15:15,Áustria Superliga,Wels,Swans Gmunden,1.85,Back Home
6987,22:00,Eua Nba,New Orleans Pelicans,Memphis Grizzlies,1.58,Back Home
6997,14:30,Alemanha Bbl,Syntainics MBC,Vechta,2.33,Back Home
